In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tsgm
import tensorflow as tf
from tensorflow import keras

In [2]:
#Save Kyoto_Gases data (2020-2100), consider the case of C1-C8 for simplicity.
Kyoto_Gases = pd.read_csv('Kyoto Gases.csv')
Kyoto_Gases = Kyoto_Gases[Kyoto_Gases['Category'].isin(['C1','C2','C3','C4','C5','C6','C7','C8'])]
mapping = {'C1':0,'C2':0,'C3':0,'C4':0,'C5':1,'C6':1,'C7':2,'C8':2}#Aggregate categories into 3 categories, with 0-2 corresponding to C1234-C78, respectively
Kyoto_Gases['Category'].replace(mapping,inplace=True)
Kyoto_Gases.reset_index(drop=True,inplace=True)
Kyoto_Gases.drop(columns=['Category_name'],inplace = True)

In [3]:
#Load a dataset of individual variables
CarbonSequestration = pd.read_csv('Carbon_Sequestration_CCS_imputed.csv')
FinalEnergy_Liquid = pd.read_csv('Final Energy_Liquids.csv')
PrimaryEnergy_Gas = pd.read_csv('Primary Energy_Gas.csv')
PrimaryEnergy_Oil = pd.read_csv('Primary Energy_Oil.csv')
PrimaryEnergy_Coal = pd.read_csv('PrimaryEnergy_Coal.csv')
PrimaryEnergy = pd.read_csv('PrimaryEnergy_imputed.csv')

In [4]:
#Get the intersection of the models and scenarios contained in each variable
Model_Scenario = Kyoto_Gases[['Model','Scenario']]
Variables = [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil,PrimaryEnergy]
for variable in Variables:
    Model_Scenario = pd.merge(Model_Scenario,variable[['Model','Scenario']],on=['Model','Scenario'],how='inner')

In [5]:
for i in range(len(Variables)):
    Variables[i] = pd.merge(Model_Scenario,Variables[i],on=['Model','Scenario'],how='inner')
for i in range(len(Variables)):
    Variables[i].drop(columns=['Category_name'],inplace = True)

In [6]:
Kyoto_Gases = pd.merge(Kyoto_Gases,Model_Scenario,on = ['Model','Scenario'],how = 'inner')

In [7]:
Variables.append(Kyoto_Gases)
#Variables [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil,PrimaryEnergy,Kyoto_Gases]
data_num = Variables[0].shape[0]

In [8]:
#Generate feature matrices, the values of each variable during 2020-2100. 9 time steps, 7 features
#data_num is the amount of data
X = np.zeros((data_num,9,7))
for i in range(len(Variables)):
    Variables[i] = Variables[i].iloc[:,3:-1].values
for i in range(data_num):
    for j in range(9):
        for k in range(7):
            X[i][j][k] = (Variables[k])[i,j]

In [9]:
Y = Kyoto_Gases['Category'].values

In [10]:
#Separate datasets by category. But the training process does not distinguish between categories like VAE
C1234_DataSet = X[Y == 0]
C56_DataSet = X[Y == 1]
C78_DataSet = X[Y == 2]

In [11]:
#Define the model structure for C1234
architecture1 = tsgm.models.zoo["vae_conv5"](9, 7, 10)#Latent Dim = 10,number of features = 7
encoder1, decoder1 = architecture1.encoder, architecture1.decoder

In [12]:
#Standardize the data and compress it to (0, 1).
scaler_C1234 = tsgm.utils.TSFeatureWiseScaler((0,1))        
scaled_C1234_data = scaler_C1234.fit_transform(C1234_DataSet)

In [ ]:
#Define an optimization strategy
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=200,
    decay_rate=0.98)
vae_C1234 = tsgm.models.cvae.BetaVAE(encoder1, decoder1)
vae_C1234.compile(optimizer=keras.optimizers.Adam(lr_schedule))
vae_C1234.fit(scaled_C1234_data, epochs=1500, batch_size=50)

In [14]:
#Define the model structure for C56
architecture2 = tsgm.models.zoo["vae_conv5"](9, 7, 10)
encoder2, decoder2 = architecture2.encoder, architecture2.decoder
scaler_C56 = tsgm.utils.TSFeatureWiseScaler((0,1))    
scaled_C56_data = scaler_C56.fit_transform(C56_DataSet)

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=200,
    decay_rate=0.98)
vae_C56 = tsgm.models.cvae.BetaVAE(encoder2, decoder2)
vae_C56.compile(optimizer=keras.optimizers.Adam(lr_schedule))
vae_C56.fit(scaled_C56_data, epochs=1500, batch_size=30)

In [ ]:
#Define the model structure for C78
architecture3 = tsgm.models.zoo["vae_conv5"](9, 7, 10)
encoder3, decoder3 = architecture3.encoder, architecture3.decoder
scaler_C78 = tsgm.utils.TSFeatureWiseScaler((0,1))    
scaled_C78_data = scaler_C78.fit_transform(C78_DataSet)

In [ ]:
vae_C78 = tsgm.models.cvae.BetaVAE(encoder3, decoder3)
vae_C78.compile(optimizer=keras.optimizers.Adam(lr_schedule))
vae_C78.fit(scaled_C78_data, epochs=1500, batch_size=30)

In [18]:
#Set Global Random Seed
global_seed = 3
tf.random.set_seed(global_seed)
np.random.seed(global_seed)

In [19]:
#Generate data using generative models (500 for each class)
z1 = tf.random.normal((1000, 10))
z2 = tf.random.normal((1000, 10))
z3 = tf.random.normal((1000, 10))
Gen_C1234 = decoder1(z1)
Gen_C56 = decoder2(z2)
Gen_C78 = decoder3(z3)
Gen_C1234 = scaler_C1234.inverse_transform(Gen_C1234)
Gen_C56 = scaler_C56.inverse_transform(Gen_C56)
Gen_C78 = scaler_C78.inverse_transform(Gen_C78)

In [ ]:
Gen_Data = np.concatenate((Gen_C1234,Gen_C56,Gen_C78),axis=0)
Gen_Labels = np.zeros(3000)
Gen_Labels[1000:2000] = 1
Gen_Labels[2000:] = 2
Gen_Labels.astype(np.int32)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [22]:
#Compute the cumulative value of the five input variables as a feature of the real dataset
features_names = ['CarbonSequestration','FinalEnergy_Liquid','PrimaryEnergy_Coal','PrimaryEnergy_Gas','PrimaryEnergy_Oil','PrimaryEnergy']
Real_Data_Sum = np.zeros((data_num,6))
for i in range(data_num):#data_num :amount of data
    for j in range(6):#6 feature dimension
        for k in range(8):#time step
            Real_Data_Sum[i][j] += (X[i,k,j] + X[i,k+1,j]) * 5

In [23]:
Real_DataSet = pd.DataFrame(Real_Data_Sum,columns=features_names)

In [24]:
#Construct the feature matrix of the feature cumulative values of the generated dataset
Gen_Data_Sum = np.zeros((3000,6))
for i in range(3000):#3000 :amount of data
    for j in range(6):#6 ：feature dimension
        for k in range(8):#time step
            Gen_Data_Sum[i][j] += (Gen_Data[i,k,j] + Gen_Data[i,k+1,j]) * 5

In [25]:
Gen_DataSet = pd.DataFrame(Gen_Data_Sum,columns=features_names)

In [26]:
#First train the model with real datasets to predict the generated data.Cross-validation using grid search
R_G = RandomForestClassifier(random_state=42)
parameters_1 = {
    'n_estimators':[10,100,500,1000],
    'max_depth':[6,8,10,12,14,16],
    'min_samples_split':[3,4,5,6]
}

In [27]:
clf_RG = GridSearchCV(R_G,parameters_1,cv=3,n_jobs=-1,verbose=2)

In [ ]:
clf_RG.fit(Real_DataSet,Y)

In [ ]:
print('Best parameters found:')
print(clf_RG.best_params_)

In [ ]:
y_pred = clf_RG.predict(Real_DataSet)
print(f"Test results on the training dataset：")
print(classification_report(Y,y_pred,target_names=['C1234','C56','C78']))

In [ ]:
y_pred = clf_RG.predict(Gen_DataSet)
print(f"Test results on the test set：")
print(classification_report(Gen_Labels,y_pred,target_names=['C1234','C56','C78']))

In [32]:
#Use the generated dataset to train models to predict real data
G_R = RandomForestClassifier(random_state=42)
parameters_2 = {
    'n_estimators':[10,100,200,500,1000],
    'max_depth':[6,8,10,12,14,16],
    'min_samples_split':[3,4,5,6]
}

In [33]:
clf_GR = GridSearchCV(G_R,parameters_2,cv=3,n_jobs=-1,verbose=2)

In [ ]:
clf_GR.fit(Gen_DataSet,Gen_Labels)
print('Best parameters found:')
print(clf_GR.best_params_)

In [ ]:
y_pred = clf_GR.predict(Gen_DataSet)
print(f"Test results on the training dataset：")
print(classification_report(Gen_Labels,y_pred,target_names=['C1234','C56','C78']))

In [ ]:
y_pred = clf_GR.predict(Real_DataSet)
print(f"Test results on the test set：")
print(classification_report(Y,y_pred,target_names=['C1234','C56','C78']))

In [37]:
encoder1.save_weights('Policy-encoder1_weights-top7.h5')
encoder2.save_weights('Policy-encoder2_weights-top7.h5')
encoder3.save_weights('Policy-encoder3_weights-top7.h5')
decoder1.save_weights('Policy-decoder1_weights-top7.h5')
decoder2.save_weights('Policy-decoder2_weights-top7.h5')
decoder3.save_weights('Policy-decoder3_weights-top7.h5')